In [41]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys, os, random, copy, collections, time, re, argparse
import numpy as np
from pprint import pprint
from utils import common
from core import models, datasets
from core.vocabularies import WordVocabularyWithEmbedding

from utils.common import dotDict
from main import main
args = dotDict({
    'checkpoint_path': 'checkpoints/tmp',
    'mode': 'train',
    'config_path': 'configs/config',
    'cleanup': True,
    'log_file': None,
    'interactive': True,
})
manager = main(args)
sys.stdout = sys.stderr

In [43]:
config = manager.config
vocab = WordVocabularyWithEmbedding(config.embeddings, vocab_size=config.vocab_size, lowercase=config.lowercase)


TypeError: super(type, obj): obj must be an instance or subtype of type

In [36]:
%whos

Variable                      Type                           Data/Info
----------------------------------------------------------------------
WordVocabularyWithEmbedding   type                           <class 'core.vocabularies<...>VocabularyWithEmbedding'>
argparse                      module                         <module 'argparse' from '<...>/python2.7/argparse.pyc'>
args                          dotDict                        {'checkpoint_path': 'chec<...>rue, 'interactive': True}
collections                   module                         <module 'collections' fro<...>thon2.7/collections.pyc'>
common                        module                         <module 'utils.common' from 'utils/common.pyc'>
config                        recDotDict                     {u'vocab_size': 30000, u'<...>aset/test.annotated.csv'}
copy                          module                         <module 'copy' from '/hom<...>/lib/python2.7/copy.pyc'>
dataset                       MapDataset       

In [19]:
dataset_type = getattr(datasets, config.dataset_type)
dataset = dataset_type(config.dataset_path, vocab)
sources, targets = dataset.symbolized
sources = sources[:3]
sources = [' '.join([str(x) for x in s]) for s in sources]

print sources
dataset = dataset.map(lambda string: tf.string_split([string]).values)
# sources = tf.keras.preprocessing.sequence.pad_sequences(sources, padding='post', truncating='post')
# print sources 
# sources = sources.map(lambda words, size: (table.lookup(words), size))
# print sources

#print tf.data.Dataset.from_tensor_slices(sources)
#print tf.data.Dataset(sources)
#print tf.keras.preprocessing.sequence.pad_sequences(dataset.symbolized)
#tf.data.Dataset.zip(dataset.symbolized)
# for i, (source, target) in enumerate(dataset.symbolized):
#     print source, target
#     print dataset.original_sources[i], [dataset.original_sources[i][t] for t in target]
#     if dataset.targets[i] != ['-'] and not ids:
#         print dataset.texts[i]
#         print dataset.values[i].split(':')[0]
#         print i, dataset.original_sources[i], dataset.targets[i], ids
    
    

['14026 1923 4397 14026 3 5409 14032 295 14026 1923 3 14026 1923 3 1865 14026 1923 16330 3835 21511 19626 23428', '12804 3 6053 17461 4737 15143 12804 3 19460 15136 17053 4397 11691 20197 14026 438 3 9485 9867 3 2373', '12804 15144 17666 3002 22160 3 3002 17936 19376 19626 15137 7620 3248 12804 17633 8265']


In [34]:
def tokens2ids(words):
    return [vocab.token2id(w) for w in words]

infer_batch = tf.placeholder(tf.string, shape=(None,))
dataset = tf.data.Dataset.from_tensor_slices(infer_batch)
dataset = dataset.map(lambda string: tf.string_split([string]).values)
dataset = dataset.map(lambda words: (words, tf.size(words)))
dataset = dataset.map(lambda words, size: (tokens2ids(words), size))
print dataset

TypeError: 'Tensor' object is not iterable.